In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using ProfileView

using Quadrature
using Cubature
using Cuba
using FastGaussQuadrature


┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
Gtk-Message: 08:51:52.069: Failed to load module "canberra-gtk-module"
Gtk-Message: 08:51:52.070: Failed to load module "canberra-gtk-module"


### Try integration on hard problems

In [38]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

In [39]:
Ne = 500
X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)
L = LinearTransform(X)
transform!(L, X);

In [40]:
S = Storage(C.I.f, X)

Storage(20, 6, 3, ParametricFunction(ExpandedFunction(20, 6, 3, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 3), [0 0 0; 0 0 1; … ; 0 1 2; 1 0 0], [0.20649582065364197, -0.5150990160472986, 2.630096893080717, 1.13653076177397, 0.6725837371023421, -1.3126095306624133])), [1.0 1.0 … 0.5072990625969442 0.4929559989641911; 1.0 1.0 … 0.5479560380832562 0.46631087792965714; … ; 1.0 1.0 … 0.6315988561844881 0.3572195479960067; 1.0 1.0 … 0.6307319114546244 0.626494976420134], [1.0 0.5688983769651315 … 0.18666451928577027 0.4929559989641911; 1.0 0.6291983140165427 … -0.042726938268403306 0.46631087792965714; … ; 1.0 0.0021417080675083184 … 0.00645150573434833 0.3572195479960067; 1.0 0.44810339513994396 … -0.33118137699908295 0.626494976420134], [1.0 0.6316187777460647 … 0.0 0.4929559989641911; 1.0 0.6316187777460647 … 0.0 0.46631087792965714; … ; 1.0 0.6316187777460647 … 0.0 0.3572195479960067; 1.0 0.6316187777460647 … 0.0 0.626494976420134], [0.0 -

In [41]:
@time inv(F.U)

  0.000086 seconds (4 allocations: 464 bytes)


6×6 UpperTriangular{Float64,Array{Float64,2}}:
 -0.0447214  -0.183189   0.18588    -0.76248   0.00808691  -0.227487
   ⋅          0.336539  -0.0136869   1.37866  -0.0143097    0.0854616
   ⋅           ⋅        -0.332099    1.41501  -0.0405491    0.10562
   ⋅           ⋅          ⋅         -2.56478   0.0721716   -0.178011
   ⋅           ⋅          ⋅           ⋅        0.253169    -0.00497192
   ⋅           ⋅          ⋅           ⋅         ⋅           0.328432

In [42]:
F = QRscaling(S);
c̃oeff =F.U*coeff;

In [7]:
# In this version, c̃oeff is expressed in the rescaled space
NxX, Ne = size(X)
m = C.m
Nx = C.Nx
Nψ = C.Nψ
# @assert NxX == Nx "Wrong dimension of the sample X"
# @assert size(S.ψoff, 1) == Ne
# @assert size(S.ψoff, 2) == Nψ

# Output objective, gradient
xlast = view(X,NxX,:)

fill!(S.cache_integral, 0.0)

# Integrate at the same time for the objective, gradient
function integrand!(v::Vector{Float64}, t::Float64)
    @time repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t*xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @time @avx @. S.cache_dcψxdt *= S.ψoff
    @time mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    @time mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    @time vJ = view(v,1:Ne)
    @time evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    @time grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    @time v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
end

integrand! (generic function with 1 method)

In [9]:
@time integrand!(S.cache_integral, 1.0);

  0.000029 seconds (18 allocations: 17.391 KiB)
  0.000011 seconds (4 allocations: 416 bytes)
  0.000026 seconds (10 allocations: 768 bytes)
  0.000005 seconds (2 allocations: 352 bytes)
  0.000002 seconds (2 allocations: 80 bytes)
  0.000014 seconds (5 allocations: 512 bytes)
  0.000005 seconds (6 allocations: 688 bytes)
  0.000025 seconds (16 allocations: 24.203 KiB)
  0.020102 seconds (18.16 k allocations: 945.902 KiB)


In [25]:
function integrand2!(v::Vector{Float64}, t::Float64)
    @time repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t*xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @time @avx @. S.cache_dcψxdt *= S.ψoff
    @time mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    @time mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    @time vJ = view(v,1:Ne)
    @time evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    @time grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)
    
    @time @avx for j=2:Nψ+1
        for i=1:Ne
            v[(j-1)*Ne+i] = S.cache_dψxd[i]*S.cache_dcψxdt[i,j-1]
        end
    end

#     @time v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
end

integrand2! (generic function with 1 method)

In [27]:
@time integrand2!(S.cache_integral, 1.0);

  0.000031 seconds (18 allocations: 17.391 KiB)
  0.000011 seconds (4 allocations: 416 bytes)
  0.000038 seconds (10 allocations: 768 bytes)
  0.000006 seconds (2 allocations: 352 bytes)
  0.000001 seconds (2 allocations: 80 bytes)
  0.000020 seconds (5 allocations: 512 bytes)
  0.000006 seconds (6 allocations: 688 bytes)
  0.000061 seconds (9 allocations: 512 bytes)
  0.001368 seconds (1.17 k allocations: 53.156 KiB)


In [15]:
cache = zeros(Ne*(Nψ+1))
@time integrand!(cache, 1.0);

  0.000142 seconds (64 allocations: 44.391 KiB)


In [14]:
norm(cache)

24.166219547946184

In [74]:
# In this version, c̃oeff is expressed in the rescaled space
NxX, Ne = size(X)
m = C.m
Nx = C.Nx
Nψ = C.Nψ
# @assert NxX == Nx "Wrong dimension of the sample X"
# @assert size(S.ψoff, 1) == Ne
# @assert size(S.ψoff, 2) == Nψ

# Output objective, gradient
xlast = view(X,NxX,:)

fill!(S.cache_integral, 0.0)

# Integrate at the same time for the objective, gradient
function integrand!(v::Vector{Float64}, t::Float64)
    repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t*xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @avx @. S.cache_dcψxdt *= S.ψoff
    mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    vJ = view(v,1:Ne)
    evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
end

# Integrate at the same time for the objective, gradient
function integrandvector_hquadrature!(t, v)
    repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t *xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @avx @. S.cache_dcψxdt *= S.ψoff
    mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    vJ = view(v,1:Ne)
    evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
    nothing
end

# Integrate at the same time for the objective, gradient
function integrandtest!(t, v)
    repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @avx @. S.cache_dcψxdt *= S.ψoff
    mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    vJ = view(v,1:Ne)
    evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
    v
end

integrandtest! (generic function with 2 methods)

In [7]:
### Gauss-Legrendre nodes
nodes, weights = gausslegendre( 20 )

([-0.993128599185095, -0.9639719272779138, -0.912234428251326, -0.8391169718222188, -0.7463319064601508, -0.636053680726515, -0.5108670019508271, -0.37370608871541955, -0.22778585114164507, -0.07652652113349734, 0.07652652113349734, 0.22778585114164507, 0.37370608871541955, 0.5108670019508271, 0.636053680726515, 0.7463319064601508, 0.8391169718222188, 0.912234428251326, 0.9639719272779138, 0.993128599185095], [0.01761400713915266, 0.040601429800386765, 0.06267204833410911, 0.0832767415767049, 0.1019301198172404, 0.11819453196151829, 0.13168863844917655, 0.142096109318382, 0.14917298647260377, 0.15275338713072584, 0.15275338713072584, 0.14917298647260377, 0.142096109318382, 0.13168863844917655, 0.11819453196151829, 0.1019301198172404, 0.0832767415767049, 0.06267204833410911, 0.040601429800386765, 0.01761400713915266])

In [91]:
# In this version, c̃oeff is expressed in the rescaled space
NxX, Ne = size(X)
m = C.m
Nx = C.Nx
Nψ = C.Nψ
# @assert NxX == Nx "Wrong dimension of the sample X"
# @assert size(S.ψoff, 1) == Ne
# @assert size(S.ψoff, 2) == Nψ

# Output objective, gradient
xlast = view(X,NxX,:)

fill!(S.cache_integral, 0.0)

# Integrate at the same time for the objective, gradient
function integrandtime!(v::Vector{Float64}, t::Float64)
    @time repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t*xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @time @avx @. S.cache_dcψxdt *= S.ψoff
    @time mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    @time mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    vJ = view(v,1:Ne)
    @time evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    @time grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    @time v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
end

# Integrate at the same time for the objective, gradient
function integrand!(v::Vector{Float64}, t::Float64)
    repeated_grad_xk_basis!(S.cache_dcψxdt, S.cache_gradxd, C.I.f.f, t*xlast)

     # This computing is also reused in the computation of the gradient, no interest to skip it
    @avx @. S.cache_dcψxdt *= S.ψoff
    mul!(S.cache_dcψxdt, S.cache_dcψxdt, F.Uinv)
    mul!(S.cache_dψxd, S.cache_dcψxdt, c̃oeff)

    # Integration for J̃
    vJ = view(v,1:Ne)
    evaluate!(vJ, C.I.g, S.cache_dψxd)

    # Integration for dcJ̃
    grad_x!(S.cache_dψxd, C.I.g, S.cache_dψxd)

    v[Ne+1:Ne+Ne*Nψ] .= reshape(S.cache_dψxd .* S.cache_dcψxdt , (Ne*Nψ))
end

integrand! (generic function with 1 method)

In [93]:
function timing()
    @btime integrand!(S.cache_integral, 1.0)
end
timing()

  35.858 μs (65 allocations: 44.56 KiB)


3000-element view(::Array{Float64,1}, 501:3500) with eltype Float64:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0014639519003089361
 -0.0014020542281708217
  0.000714459069375454
  0.0014704695029350407
  0.0003996756132306676
 -1.2659076863160978e-8
  0.0005073921323785968
  0.00132869694334589
  0.0015252389865262784
 -0.0005671002110116636
  0.0012680832652247045
  0.0008901020816033351

In [94]:
@profview

LoadError: LoadError: UndefVarError: @profview not defined
in expression starting at In[94]:1

In [89]:
@time integrand!(S.cache_integral, 1.0);

  0.000028 seconds (18 allocations: 17.391 KiB)
  0.000010 seconds (4 allocations: 416 bytes)
  0.000040 seconds (10 allocations: 768 bytes)
  0.000006 seconds (2 allocations: 352 bytes)
  0.000001 seconds (2 allocations: 80 bytes)
  0.000020 seconds (6 allocations: 560 bytes)
  0.000007 seconds (6 allocations: 688 bytes)
  0.000039 seconds (16 allocations: 24.203 KiB)
  0.000974 seconds (1.21 k allocations: 76.594 KiB)


In [86]:
@code_lowered integrand!(S.cache_integral, 1.0)

CodeInfo(
1 ──        Core.NewvarNode(:(stats@_4))
│           Core.NewvarNode(:(elapsedtime@_5))
│           Core.NewvarNode(:(val@_6))
│           Core.NewvarNode(:(diff@_7))
│           Core.NewvarNode(:(stats@_8))
│           Core.NewvarNode(:(elapsedtime@_9))
│           Core.NewvarNode(:(val@_10))
│           Core.NewvarNode(:(diff@_11))
│           Core.NewvarNode(:(stats@_12))
│           Core.NewvarNode(:(elapsedtime@_13))
│           Core.NewvarNode(:(val@_14))
│           Core.NewvarNode(:(diff@_15))
│           Core.NewvarNode(:(stats@_16))
│           Core.NewvarNode(:(elapsedtime@_17))
│           Core.NewvarNode(:(val@_18))
│           Core.NewvarNode(:(diff@_19))
│           Core.NewvarNode(:(stats@_20))
│           Core.NewvarNode(:(elapsedtime@_21))
│           Core.NewvarNode(:(val@_22))
│           Core.NewvarNode(:(diff@_23))
│           Core.NewvarNode(:(stats@_24))
│           Core.NewvarNode(:(elapsedtime@_25))
│           Core.NewvarNode(:(val@_26))
│          

In [75]:
@time cuhre( (x,f) -> f[1] = log(x[1])/sqrt(x[1]))

  0.064386 seconds (184.92 k allocations: 9.566 MiB)


Component:
 1: -4.000000355067185 ± 0.00033954840286260385 (prob.: 0.0)
Integrand evaluations: 5915
Number of subregions:  46
Note: The desired accuracy was reached

In [76]:
@time quadgk( x-> log(x)/sqrt(x), 0.0, 1.0)

  0.068890 seconds (161.03 k allocations: 8.584 MiB)


(-3.999999968428393, 4.547916264758092e-8)

In [77]:
cache = zeros(Ne*(Nψ+1));

In [79]:
fill!(S.cache_integral, 0.0)
@time quadgk!(integrand!, S.cache_integral, 0.0, 1.0; rtol = 1e-3)#, order = 3)#; order = 9, rtol = 1e-10)

  0.001279 seconds (989 allocations: 885.938 KiB)


([1.1006899299918569, 1.1349877566278743, 1.059634699247715, 1.112280250700692, 0.9806277420547527, 1.086989863707072, 1.123803295997763, 1.1410202353540442, 1.1310341868512825, 1.1290528114182772  …  0.0007336777534248057, 0.0012112788263278545, 0.001114759629349068, 0.00041279631767179663, 0.0006804819709191911, 0.0010952914313967599, 0.0012954699332062192, 0.00014197217873530133, 0.0013566537695462871, 0.0008277339856122307], 7.897833729966686e-6)

In [80]:
fill!(S.cache_integral, 0.0)
@time hquadrature_v(Ne*(Nψ+1), integrandtest!, zeros(Ne), X[; reltol = 1e-3)

LoadError: ArgumentError: quadrature routines are for 1d only

In [68]:
fill!(S.cache_integral, 0.0)
@time pquadrature(Ne*(Nψ+1), integrandvector_hquadrature!, 0.0, 1.0; reltol = 1e-3)

  0.002909 seconds (2.20 k allocations: 1.488 MiB)


([1.100689929991857, 1.1349877566278743, 1.059634699247715, 1.112280250700692, 0.9806277420547297, 1.0869898637070718, 1.1238032959977629, 1.1410202353540442, 1.1310341868512828, 1.1290528114182772  …  0.0007336777534248054, 0.0012112788263278548, 0.001114759629349068, 0.00041279631934728185, 0.0006804819709191911, 0.0010952914313967597, 0.0012954699332062192, 0.0001419721787353013, 0.0013566537695462876, 0.0008277339856122307], [4.440892098500626e-16, 0.0, 0.0, 0.0, 4.9152903969229556e-12, 0.0, 2.220446049250313e-16, 0.0, 2.220446049250313e-16, 2.220446049250313e-16  …  2.168404344971009e-19, 4.336808689942018e-19, 4.401860820291148e-17, 1.1550325095237984e-11, 0.0, 2.168404344971009e-19, 2.168404344971009e-19, 0.0, 2.168404344971009e-19, 0.0])

In [56]:
S = Storage(C.I.f, X)
@time result, err = suave(integrandvector!, 1, Ne*(Nψ+1))

  0.070020 seconds (94.92 k allocations: 4.866 MiB)


Components:
    1: 6.93793323611145e-310 ± 5.84152943e-316 (prob.: 1.01896012e-315)
    2: 6.93793323611145e-310 ± 0.0 (prob.: 3.3336e-319)
    3: 1.018681386e-315 ± 0.0 (prob.: 6.9379332360158e-310)
    4: 1.018681386e-315 ± 2.1219957905e-314 (prob.: 1.01881996e-315)
    5: 1.018681703e-315 ± 1.018820515e-315 (prob.: 0.0)
    6: 8.487983165e-314 ± 8.487983165e-314 (prob.: 0.0)
    7: 1.018719123e-315 ± 1.01881854e-315 (prob.: NaN)
    8: 0.0 ± 4.71936e-319 (prob.: 1.8326974e-316)
    9: 1.01868281e-315 ± 6.9379332360158e-310 (prob.: NaN)
   10: 2.121995791e-314 ± 6.9379332360158e-310 (prob.: 1.018957826e-315)
   11: 0.0 ± 0.0 (prob.: 1.018961067e-315)
   12: NaN ± 0.0 (prob.: 6.93793295076997e-310)
   13: 9.6812768e-316 ± 5.6374412e-316 (prob.: 9.9797624e-316)
   14: 3.1985776235367574e-308 ± 5.0e-324 (prob.: 1.0189598e-315)
   15: 1.0186821e-315 ± 2.57e-322 (prob.: 1.38530251221589e-309)
   16: 8.487983164e-314 ± 4.243991582e-314 (prob.: 1.020922705e-315)
   17: 1.018681663e-315 ± 0.

In [27]:
function fun_vec(x,f)
           f[1,:] .= 1.0
           for j in 1:size(x,2)
               for i in 1:size(x, 1)
                   f[1, j] *= cos(x[i, j])
               end
           end
end

fun_vec (generic function with 1 method)

In [29]:
cuhre(fun_vec, 10, nvec = 1000)

Component:
 1: 0.17798706658707045 ± 1.070799596273229e-6 (prob.: 0.2438374079277991)
Integrand evaluations: 7815
Number of subregions:  2
Note: The desired accuracy was reached

In [61]:
function timing()
    @btime begin
        fill!(S.cache_integral, 0.0)
        quadgk!(integrand!, S.cache_integral, -1.0, 1.0; rtol = 1e-1, order = 2)
    end
    
    @btime begin
        
        fill!(S.cache_integral, 0.0)
        @inbounds for i=1:20
            integrand!(cache, nodes[i])
            axpy!(weights[i], cache, S.cache_integral)
        end
    end
end

timing (generic function with 1 method)

In [62]:
S.cache

  199.400 μs (344 allocations: 414.69 KiB)
  719.699 μs (1341 allocations: 892.05 KiB)


In [81]:
fill!(S.cache_integral, 0.0)
quadgk!(integrandvector!, S.cache_integral, zeros(Ne), ones(Ne); rtol = 1e-3)#, order = 2)#; order = 9, rtol = 1e-10)

LoadError: MethodError: no method matching one(::Type{Array{Float64,1}})
Closest candidates are:
  one(!Matched::Type{Missing}) at missing.jl:103
  one(!Matched::BitArray{2}) at bitarray.jl:426
  one(!Matched::Missing) at missing.jl:100
  ...

In [82]:
?quadgk!

search: quadgk! quadgk QuadGK QuadGKJL



```
quadgk!(f!, result, a,b,c...; rtol=sqrt(eps), atol=0, maxevals=10^7, order=7, norm=norm)
```

Like `quadgk`, but make use of in-place operations for array-valued integrands (or other mutable types supporting in-place operations).  In particular, there are two differences from `quadgk`:

1. The function `f!` should be of the form `f!(y, x) = y .= f(x)`.  That is, it writes the return value of the integand `f(x)` in-place into its first argument `y`.   (The return value of `f!` is ignored.)
2. Like `quadgk`, the return value is a tuple `(I,E)` of the estimated integral `I` and the estimated error `E`.   However, in `quadgk!` the estimated integral is written in-place into the `result` argument, so that `I === result`.

Otherwise, the behavior is identical to `quadgk`.

For integrands whose values are *small* arrays whose length is known at compile-time, it is usually more efficient to use `quadgk` and modify your integrand to return an `SVector` from the [StaticArrays.jl package](https://github.com/JuliaArrays/StaticArrays.jl).


In [66]:
norm(S.cache_integral)

23.679104302423468

In [55]:
@time qrnegative_log_likelihood!(0.0, zeros(Nψ), c̃oeff, F, S, C, X)

  0.001344 seconds (316 allocations: 878.938 KiB)


434.37814763630524

### Toy problem

In [43]:
function f(dx, x, p) 
    dx[1] = sin(x)
    dx[2] = cos(x)*exp(-x^2/2)
end


  0.000083 seconds (103 allocations: 6.328 KiB)


QuadratureProblem. In-place: true


In [48]:
@btime begin 
    prob = QuadratureProblem(f, 1.0, 3.0; nout = 2)
    sol = solve(prob, CubatureJLh(),reltol=1e-3,abstol=1e-3)
end

  16.211 μs (147 allocations: 8.34 KiB)


u: 2-element Array{Float64,1}:
 1.5302948024685854
 0.026603108253927005

In [49]:
sol = solve(prob, CubatureJLh(),reltol=1e-3,abstol=1e-3)

u: 2-element Array{Float64,1}:
 1.5302948024685854
 0.026603108253927005

In [28]:
quadgk(x->sin(x), 1.0, 3.0, rtol = 1e-3, atol = 1e-3)

(1.530294802468585, 1.9984014443252818e-15)

In [ ]:
function timing()
    @btime begin 
        
        
    end

In [75]:
function f(dx,x,p)
  Threads.@threads for i in 1:size(x,2)
    dx[i] = sum(sin.(@view(x[:,i])))
  end
end
prob = QuadratureProblem(f,ones(2),3ones(2),batch=2)
sol = solve(prob,CubatureJLh(),reltol=1e-3,abstol=1e-3)

u: 1-element Array{Float64,1}:
 6.121179417774831